In [1]:
!git clone https://github.com/danibachar/idc_CANLab.git

fatal: destination path 'idc_CANLab' already exists and is not an empty directory.


In [2]:
!pip3 install -r idc_CANLab/requirements.txt

In [3]:
from google.colab import drive
from google.colab import files
    
from idc_CANLab.functions import utils,plots,consts, summary, backup # Our functionalities

from ipywidgets import interact, interactive, fixed, interact_manual#, FileUpload
import ipywidgets as widgets
from IPython.display import display
from IPython.display import FileLink, FileLinks
# Data manipulation and calculation packages
import pandas as pd
import numpy as np
import os

/usr/local/lib/python3.7/site-packages/google/colab/data_table.py:30: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


# Data Structures

In [4]:
dfs = {}
data_frame_names = []
cleaning_dfs = {}
cleaning_file_names = []

# Please enter your name, and a name for your experiment
## This will help us and you to build more meaningfull file names for your exported data and graphs

In [5]:
researcher_name = widgets.Text(
    value=utils.get_random_string(10),
    placeholder='Type your name'
)
project_name = widgets.Text(
    value=utils.get_random_string(6),
    placeholder='Type your project name'
)

names = widgets.VBox([researcher_name,project_name])
names

# Settings up environment variables to config your research environment

In [6]:
os.environ[consts.PROJECT_NAME_ENV_VAR]=project_name.value
os.environ[consts.USERNAME_ENV_VAR]=researcher_name.value

# 1) File Uploads
## Please note we currently support only CSV files.
##    If your'e using Excel or other file formats please convert to CSV
###  Excel to CSV - https://knowledgebase.constantcontact.com/articles/KnowledgeBase/6409-saving-an-excel-file-as-a-csv-file?lang=en_US
### EDF to CSV - https://emotiv.gitbook.io/emotivpro/convert_edf_to_csv

## 1.1) Choose Primary file to upload - Only one file!

## If you alerady uploaded the file you can select it 

In [10]:
def choose_filename(filename):
  return filename
primary_file_name_dropdown = widgets.Dropdown(
    options=utils.list_all_files_in_dir("."),
    description='File name:',
)
# primary_file_name_dropdown
interact(choose_filename, filename=primary_file_name_dropdown)

interactive(children=(Dropdown(description='File name:', options=('./idc_CANLab/examples/gal_data/bad_experime…

<function __main__.choose_filename(filename)>

In [11]:
file_name = primary_file_name_dropdown.value
dfs['primary'] = pd.read_csv(file_name)
data_frame_names = list(dfs.keys())

## If you have not uploaded the file already, please upload it
## If you used the 2 boxes above make sure to SKIP these 2 boxes, or else you will override your data

In [9]:
uploaded = files.upload()
assert len(uploaded.items()) == 1

KeyboardInterrupt: 

In [ ]:
for file_name, byte_file in uploaded.items():
  dfs['primary'] = pd.read_csv(io.StringIO(byte_file.decode("utf-8")))
data_frame_names = list(dfs.keys())

## 1.2) Choose Other files to upload, files we wish to merge to the primary file, like demographic data, metadata on the trial and others
## If you alerady uploaded the file you can select it 

In [12]:
def choose_filename(filename):
  return filename
secondary_files_name_dropdown = widgets.SelectMultiple(
    options=utils.list_all_files_in_dir("."),
    description='File name:',
)

interact(choose_filename, filename=secondary_files_name_dropdown)

interactive(children=(SelectMultiple(description='File name:', options=('./idc_CANLab/examples/gal_data/bad_ex…

<function __main__.choose_filename(filename)>

In [13]:
scondary_file_names = list(secondary_files_name_dropdown.value)
for file_name in scondary_file_names:
  dfs[file_name] = pd.read_csv(file_name)
data_frame_names = list(dfs.keys())

## If you have not uploaded the file already, please upload it
## If you used the 2 boxes above please skip these 2

In [ ]:
merged_uploaded = files.upload()

In [ ]:
for file_name, byte_file in merged_uploaded.items():
  data_frame_names.append(file_name)
  dfs[file_name] = pd.read_csv(io.StringIO(byte_file.decode("utf-8")))

## 1.3) Data Sumarization

In [ ]:
print(summary.info_dfs(list(dfs.values())))

# 2) Merging Extra data from external files - (for example memory span, age, etc)

### Please note that if you wish to look at the raw data before adding more from external sources, all you need to do is just skip the following box and move to the plotting part

### Please note, it is recommended to read a about JOIN and ANTI_JOIN opparations in SQL and Python Pandas to fully understand what we are doing here, although not mandatory. We would try and make the process as simple as possible


## 2.1) For each file we wish to merge, select the columns you wish to merge by with the primary file, these columns will be used as unique key and the recommendation will be the User Id/Session Label/Subject and Trial_Id/Trial Label.

In [14]:
def choose_colums(colums):
  return colums

# Widget that will be shwon at the bottom, after running the box
external_merge_columns_dropdown = []
for i in range(1,len(set(data_frame_names))):
  if data_frame_names[i] == "primary":
    continue
  w1 = widgets.SelectMultiple(
      options=dfs[data_frame_names[i]].columns,
      description=data_frame_names[i]
  )
  w2 = widgets.SelectMultiple(
      options=dfs[data_frame_names[0]].columns,
      description=data_frame_names[0]
  )
  external_merge_columns_dropdown.append(widgets.HBox([w1,w2]))
  
columns_map = widgets.VBox(external_merge_columns_dropdown)
columns_map

## 2.2) Merging is done automatically

In [15]:
merged_df = dfs[data_frame_names[0]].copy()
for i in range(1,len(set(data_frame_names))):
  child = columns_map.children[i-1]
  left = list(child.children[1].value)
  right = list(child.children[0].value)
  merged_df = merged_df.merge(dfs[data_frame_names[i]], left_on=left, right_on=right, how='inner')


## 2.3) Data Sumarization

In [16]:

print(summary.info_dfs(list(dfs.values())))
for sum_url in summary.describe_dfs(list(dfs.values())):
    print(sum_url)

https://storage.googleapis.com/outliers/vwmmsxanip/kstuzc/info_07-18-2020-17%3A22%3A38.txt
https://storage.googleapis.com/outliers/object_vwmmsxanip/kstuzc/descibe_07-18-2020-17%3A22%3A40.csv
https://storage.googleapis.com/outliers/numeric_vwmmsxanip/kstuzc/descibe_07-18-2020-17%3A22%3A40.csv
https://storage.googleapis.com/outliers/numeric_vwmmsxanip/kstuzc/descibe_07-18-2020-17%3A22%3A42.csv


# 3) Droping unwanted colums
## Here we suggest removing uneeded columns from our primary merged file, this could help both better performance and understanding of the data 


## 3.1) Choose unwanted colums

In [17]:
def choose_colums(colums):
  return colums
# Widget that will be shwon at the bottom, after running the box
colums_dropdown = widgets.SelectMultiple(
    options=merged_df.columns,
    description='Select colums to drop:',
)

interact(choose_colums, colums=colums_dropdown)

interactive(children=(SelectMultiple(description='Select colums to drop:', options=('RECORDING_SESSION_LABEL',…

<function __main__.choose_colums(colums)>

## 3.2) Dropping the columns

In [18]:
colums_to_drop = list(colums_dropdown.value)
merged_df = merged_df.drop(columns=colums_to_drop)

# Flattening 
for col in merged_df.columns:
  if merged_df[col].dtype == object:
    merged_df[col] = merged_df[col].astype('category')

## 3.3) Data Sumarization

In [21]:
print(summary.info_dfs([merged_df]))
for sum_url in summary.describe_dfs([merged_df]):
    print(sum_url)


https://storage.googleapis.com/outliers/vwmmsxanip/kstuzc/info_07-18-2020-17%3A25%3A52.txt
https://storage.googleapis.com/outliers/category_vwmmsxanip/kstuzc/descibe_07-18-2020-17%3A25%3A52.csv
https://storage.googleapis.com/outliers/numeric_vwmmsxanip/kstuzc/descibe_07-18-2020-17%3A25%3A52.csv


## 3.4) Choose columns to convert to categorical, for example subject_id, trial_id, memory span and others should be categorical
## Wants to convert some numeric values for categorical? Do it now!

In [22]:
def choose_colums(colums):
  return colums
# Widget that will be shwon at the bottom, after running the box
categorical_colums_dropdown = widgets.SelectMultiple(
    options=merged_df.columns,
    description='Select colums to drop:',
)

interact(choose_colums, colums=categorical_colums_dropdown)

interactive(children=(SelectMultiple(description='Select colums to drop:', options=('RECORDING_SESSION_LABEL',…

<function __main__.choose_colums(colums)>

In [23]:
colums_to_convert_to_categorical = list(categorical_colums_dropdown.value)
for col in colums_to_convert_to_categorical:
  merged_df[col] = merged_df[col].astype('category')

# 4) Ploting the raw data we have so far

## In the next few boxes we are going through a process of plotting the trials a certain Subject was going through

## 4.1) Per Subject Ploting

### 4.1.1) Select the name of the column you wish to group by, usually the subject or trial columns

In [24]:
def col_name(column_name):
  return column_name

groupped_column_dropdown = widgets.Dropdown(
    options=merged_df.columns,
    description='Subject:',
)

interact(col_name, column_name=groupped_column_dropdown)

interactive(children=(Dropdown(description='Subject:', options=('RECORDING_SESSION_LABEL', 'TRIAL_INDEX', 'BIN…

<function __main__.col_name(column_name)>

## 4.1.2) Select the desire columns you wish to have as your y and x axis.
### For y, usually the column that follow the eye gaze on the Target, but could also be for the Competitor or one of the fillers
### For x, usually the time column

In [25]:
def choose_axis(axis):
  return axis

y_column_dropdown = widgets.Dropdown(
    options=merged_df.columns,
    description='Select `Y` axis column name:',
)
x_column_dropdown = widgets.Dropdown(
    options=merged_df.columns,
    description='Select `X` axis column name:',
)

interact(choose_axis, axis=y_column_dropdown)
interact(choose_axis, axis=x_column_dropdown)

interactive(children=(Dropdown(description='Select `Y` axis column name:', options=('RECORDING_SESSION_LABEL',…

interactive(children=(Dropdown(description='Select `X` axis column name:', options=('RECORDING_SESSION_LABEL',…

<function __main__.choose_axis(axis)>

## 4.1.3) Select the different features (e.g memory span, critical, load, noise level) you want to create the grid by

In [26]:
def choose_colums(colums):
  return colums
# Widget that will be shwon at the bottom, after running the box
features_column_dropdown = widgets.SelectMultiple(
    options=merged_df.columns,
    description='Select colums to drop:',
)

interact(choose_colums, colums=features_column_dropdown)

interactive(children=(SelectMultiple(description='Select colums to drop:', options=('RECORDING_SESSION_LABEL',…

<function __main__.choose_colums(colums)>

## 4.1.4) Plot!!

In [27]:
groupping_col_name = groupped_column_dropdown.value
y_name = y_column_dropdown.value
x_name = x_column_dropdown.value
grid_features = list(features_column_dropdown.value)

_, url = plots.plots_by_group_and_features(merged_df, groupping_col_name, y_name, x_name, grid_features)
print("plots_by_group_and_features - ",url)
_, url = plots.plot_general_avg_grid(merged_df, y_name, x_name, grid_features)
print("plot_general_avg_grid - ",url)
_, url = plots.plot_general_avg(merged_df, y_name, x_name)
print("plot_general_avg - ",url)

plots_by_group_and_features -  https://storage.googleapis.com/outliers/vwmmsxanip/kstuzc/groups_by_features_07-18-2020-17%3A26%3A51.html
plot_general_avg_grid -  https://storage.googleapis.com/outliers/vwmmsxanip/kstuzc/general_avg_grid_07-18-2020-17%3A27%3A31.html
plot_general_avg -  https://storage.googleapis.com/outliers/vwmmsxanip/kstuzc/general_avg_07-18-2020-17%3A27%3A38.html


# 5) Filtering and Data Cleaning
## You can repeat the folowing steps in order to clean different values from different colmuns

## 5.1) Drop rows with Nan/Null values

In [28]:
merged_df = merged_df.dropna()

## 5.2) Choose Column to filter by

In [29]:
def col_name(column_name):
  return column_name

filter_column_dropdown = widgets.Dropdown(
    options=merged_df.columns,
    description='Col Name:',
)

interact(col_name, column_name=filter_column_dropdown)

interactive(children=(Dropdown(description='Col Name:', options=('RECORDING_SESSION_LABEL', 'TRIAL_INDEX', 'BI…

<function __main__.col_name(column_name)>

## 5.3) Choose value to remove from the data

In [30]:
col_name = filter_column_dropdown.value
values = list(merged_df[col_name].unique())
def value_name(value):
  return value

value_name_dropdown = widgets.Dropdown(
    options=values,
    description='Value Name:',
)

interact(value_name, value=value_name_dropdown)

interactive(children=(Dropdown(description='Value Name:', options=('n', 'y'), value='n'), Output()), _dom_clas…

<function __main__.value_name(value)>

## 5.4) Filter that value out!

In [31]:
value = value_name_dropdown.value
selector = (merged_df[col_name] != value)
merged_df = merged_df[selector]

## 5.5) Data Summarization and plotting

In [44]:
print(summary.info_dfs([merged_df]))
for sum_url in summary.describe_dfs([merged_df]):
    print(sum_url)

_, url = plots.plots_by_group_and_features(merged_df, groupping_col_name, y_name, x_name, grid_features)
print("plots_by_group_and_features - ",url)
_, url = plots.plot_general_avg_grid(merged_df, y_name, x_name, grid_features)
print("plot_general_avg_grid - ",url)
_, url = plots.plot_general_avg(merged_df, y_name, x_name)
print("plot_general_avg - ",url)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
https://storage.googleapis.com/outliers/vwmmsxanip/kstuzc/info_07-18-2020-17%3A48%3A26.txt
https://storage.googleapis.com/outliers/category_vwmmsxanip/kstuzc/descibe_07-18-2020-17%3A48%3A27.csv
https://storage.googleapis.com/outliers/numeric_vwmmsxanip/kstuzc/descibe_07-18-2020-17%3A48%3A27.csv
plots_by_group_and_features -  https://storage.googleapis.com/outliers/vwmmsxanip/kstuzc/groups_by_features_07-18-2020-17%3A48%3A30.html
plot_general_avg_grid -  https://storage.googleapis.com/outliers/vwmmsxanip/kstuzc/general_avg_grid_07-18-2020-17%3A48%3A48.html
plot_general_avg -  https://storage.googleapis.com/outliers/vwmmsxanip/kstuzc/general_avg_07-18-2020-17%3A48%3A56.html


## 5.6) Filter according to exteral files.
## Here we will use the idea behind anti join to drop certain rows accroding to files you have prepared in advance

### 5.6.1) If you alerady uploaded the file you can select it

In [63]:
def choose_filename(filename):
  return filename
anti_join_files_name_dropdown = widgets.SelectMultiple(
    options=utils.list_all_files_in_dir("."),
    description='File name:',
)

interact(choose_filename, filename=anti_join_files_name_dropdown)

interactive(children=(SelectMultiple(description='File name:', options=('./idc_CANLab/examples/gal_data/bad_ex…

<function __main__.choose_filename(filename)>

In [66]:
cleaning_dfs = {}
anti_join_file_names = list(anti_join_files_name_dropdown.value)
for file_name in anti_join_file_names:
  c_df = pd.read_csv(file_name)
  for col in c_df.columns:
    if col in merged_df.columns:
      c_df[col] = c_df[col].astype(merged_df[col].dtype)
  cleaning_dfs[file_name] = c_df
cleaning_file_names = list(cleaning_dfs.keys())

### 5.6.2) If you have not uploaded the file already, please upload it
### If you used the 2 boxes above please skip these 2

In [ ]:
anti_merge_uploaded = files.upload()

In [ ]:
for file_name, byte_file in anti_merge_uploaded.items():
  c_df = pd.read_csv(io.StringIO(byte_file.decode("utf-8")))
  for col in c_df.columns:
    if col in merged_df.colmuns:
      c_df[col] = c_df[col].astype(merged_df[col].dtype)
        
  cleaning_dfs[file_name] = c_df
cleaning_file_names = list(cleaning_dfs.keys())

### 5.6.3) Choose the unique column names you wish to anti-join.
### This columns hsould create a unique like key so we can tell our notebook to remove each row that corresponds from the file to the data we have cleaning and merged so far

In [67]:
def choose_colums(colums):
  return colums

# Widget that will be shwon at the bottom, after running the box
external_clean_columns_dropdown = []
for i in range(0,len(cleaning_file_names)):
  w1 = widgets.SelectMultiple(
      options=cleaning_dfs[cleaning_file_names[i]].columns,
      description=cleaning_file_names[i]
  )
  w2 = widgets.SelectMultiple(
      options=merged_df.columns,
      description='primary data frame'
  )
  external_clean_columns_dropdown.append(widgets.HBox([w1,w2]))
  
columns_map = widgets.VBox(external_clean_columns_dropdown)
columns_map

In [70]:
for i in range(len(cleaning_file_names)):
  # Flat columns from import files
  clean_df = cleaning_dfs[cleaning_file_names[i]]
  child = columns_map.children[i-1]
  left = list(child.children[1].value)
  right = list(child.children[0].value)
  merged_df = utils.anti_join(merged_df, clean_df, left_on=left, right_on=right)

['RECORDING_SESSION_LABEL', 'TRIAL_INDEX']
['RECORDING_SESSION_LABEL', 'TRIAL_INDEX']


### 5.6.4) Data Summarization and plotting

In [80]:
print(summary.info_dfs([merged_df]))
for sum_url in summary.describe_dfs([merged_df]):
    print(sum_url)

_, url = plots.plots_by_group_and_features(merged_df, groupping_col_name, y_name, x_name, grid_features)
print("plots_by_group_and_features - ",url)
_, url = plots.plot_general_avg_grid(merged_df, y_name, x_name, grid_features)
print("plot_general_avg_grid - ",url)
_, url = plots.plot_general_avg(merged_df, y_name, x_name)
print("plot_general_avg - ",url)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
https://storage.googleapis.com/outliers/vwmmsxanip/kstuzc/info_07-18-2020-19%3A15%3A32.txt


# 6) Saving files
## At any stage you can choose to save the current state of our manipulated data

## 6.1) Full backup

In [78]:
urls = saved_file_name = backup.save_locally_and_update(merged_df, dfs, cleaning_dfs)
for url in urls:
    print(url)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
vwmmsxanip/kstuzc/original_07-18-2020-18:28:39.csv
original_07-18-2020-18:28:39.csv
done
vwmmsxanip/kstuzc/original_07-18-2020-18:37:00.csv
original_07-18-2020-18:37:00.csv
done
vwmmsxanip/kstuzc/original_07-18-2020-18:37:00.csv
vwmmsxanip/kstuzc/primary_07-18-2020-18:37:01.csv
['https://storage.googleapis.com/outliers/vwmmsxanip/kstuzc/original_07-18-2020-18%3A28%3A39.csv', 'https://storage.googleapis.com/outliers/vwmmsxanip/kstuzc/original_07-18-2020-18%3A37%3A00.csv', 'https://storage.googleapis.com/outliers/vwmmsxanip/kstuzc/clean_07-18-2020-18%3A37%3A01.csv', 'https://storage.googleapis.com/outliers/vwmmsxanip/kstuzc/primary_07-18-2020-18%3A37%3A01.csv']
['https://storage.googleapis.com/outliers/vwmmsxanip/kstuzc/original_07-18-2020-18%3A28%3A39.csv', 'https://storage.googleapis.com/outliers/vwmmsxanip/kstuzc/original_07-18-2020-18%3A37%3A00.csv', 'https://storage.googleapis.com/outliers/vwmmsx

## 6.2) Diff backup

In [79]:
urls = saved_file_name = backup.save_locally_and_update(merged_df)
for url in urls:
    print(url)

https://storage.googleapis.com/outliers/vwmmsxanip/kstuzc/primary_07-18-2020-18%3A46%3A10.csv


In [ ]:
merged_df.